Autor: Roman Studer 

In [2]:
import pandas as pd
import numpy as np
import csv
import pickle
from pathlib import Path

# Product Reduction using a Rating
------
This Notebook is used to create a List of all products, rate them and eliminate up to 80% of the "weakest" products. (Rating should not be used for a recommender at a later state)

In [3]:
#load first few roads to get an overview of the data
overview = pd.read_csv('Recommender4Retail.csv', nrows = 10)
overview.head()
df = pd.read_csv('Recommender4Retail.csv', nrows = 1_000_000)

### Extract a list of Products and Orders out of a DataFrame
----
The following code-blocks are used to extract all unique values for a certain column (in our case `product_name`). 
The difficulty here is that the dataset is to large to be loaded at once. Because the extraction of the unique values for a large dataset takes some time I decided to save the created lists as a pickle file to be able to load them in after a restart of the notebook.

In [3]:
def extract_unique_values(data,column, chunksize=None):
    """
    Extracts the unique values for a certain column in a dataframe
    
    :param str, data is the path to a csv
    :column str, column to be filtered
    :chunksize int, defines the size of a batch to load
    
    :return list, of all unique values
    """
    products = []
    
    for chunk in pd.read_csv(data, chunksize=chunksize):
        
        # could be optimized 
        t = chunk[column].value_counts()
        t = pd.DataFrame(t).reset_index()
        t = t.rename(columns={'index': 'product'})
        
        for val in t['product'].values.tolist():
            products.append(val)
    list_set = set(products)
    unique_val = list(list_set)
    
    return unique_val

#### Use function to create a list of unique products and unique orders

In [4]:
if Path('product_ids.pkl').is_file():
    pass
else:
    products = extract_unique_values('Recommender4Retail.csv', 'product_name', chunksize=10_000)
    with open('product_ids.pkl', 'wb') as f:
        pickle.dump(products, f)

# load saved data to 
with open('product_ids.pkl', 'rb') as f:
    products = pickle.load(f);

In [5]:
if Path('order_ids.pkl').is_file():
    pass
else:
    orders = extract_unique_values('Recommender4Retail.csv', 'order_id', chunksize=10_000)
    with open('order_ids.pkl', 'wb') as f:
        pickle.dump(orders, f)
        
# load saved data to 
with open('order_ids.pkl', 'rb') as f:
    orders = pickle.load(f);

In [6]:
f'Total number of Products in Recommender4Retail.csv: {len(products)}' 
# Products: 49_685

'Total number of Products in Recommender4Retail.csv: 31330'

In [7]:
f'Total number of Customers in Recommender4Reteail.csv: {len(orders)}' 
# Orders: 3_346_083

'Total number of Customers in Recommender4Reteail.csv: 3346083'

### Create DataFrame as a basis for a rating
----
The code above created two lists called `products` and `orders` representing all products and orders in the dataset. With this information, we can calculate a rating for every product. The function `calc_product_order_reorder_df` creates a DataFrame containing columns which contain information that can be used to create a rating of the product.

- `index` = product id
- `n_orders` = number of orders for the product
- `n_users` = number of users that ordered the product
- `n_reorders` = number of reorders for the product

In [8]:
def calc_product_order_reorder_count():
    chunks = pd.read_csv("Recommender4Retail.csv",chunksize=10_000)
    subsets = [chunk.groupby('product_name').agg({'product_name':'count',
                                                'user_id': 'nunique',
                                                'reordered':'sum'}) for chunk in chunks]

    df = pd.concat(subsets).groupby(level=0).sum()
    
    df.reset_index(inplace=True, drop=True)
    df.rename(columns={"product_name": "n_orders", "user_id": "n_users", 'reordered': "n_reorders"}, inplace=True)
    return df

In [9]:
df = calc_product_order_reorder_count()

In [10]:
df.head()

,n_orders,n_users,n_reorders
0,25,13,12
1,2,2,0
2,26,5,21
3,5,3,2
4,6,3,3


In [11]:
# sanity check: 
print(f'Number of Products in list: {len(df)}') # should be 49685 -> correct

Number of Products in list: 31330


### Calcuate the Support of the Products and implement it into a rating formula
-----

The Apriori - Algoritmn works with two probabilistic measurements which rate the assoziationrules: Support and Confidence. The support represents the probability of an item being in a basket. We use the basic idea of the support to calculate a Rating for every product containing information about how often the product has been bought, how many customers bought the product, and how many of those customers reordered the product. This gives us the "rating" formula below:

#### Mathematical Approach
To callculate the support we need a Multiset $$X = \text{Product for every transaction}$$ containing all ordered products. 

The formula to calculate the Support can be written as follows:
$$
support(x\in X) =  \frac{|\forall x \in X|}{|X|}
$$

Or in writing:
$$
support(\text{Product}) =  \frac{\text{Number of Buys of the Product}}{\text{Total number of transactions}}
$$

Rating formula:
$$
rating(\text{Product}) =  \text{normalized}( \frac{\text{Number of Buys of the Product}}{\text{Total number of transactions}}+\frac{\text{Number of Customers for the Product}}{\text{Total number of Customers}}+\frac{\text{Number of Reorders of the Product}}{\text{Total number of Reorders}} )
$$

In [12]:
# calculate rating
n_orders = sum(df['n_orders'])
n_customers = sum(df['n_users'])
n_reorders = sum(df['n_reorders'])
df['rating']= df['n_orders']/n_orders + df['n_users']/n_orders + df['n_reorders']/n_reorders

# normalize rating
df['rating'] = (df['rating']-df['rating'].min())/(df['rating'].max()-df['rating'].min())

df.head()

,n_orders,n_users,n_reorders,rating
0,25,13,12,0.001585
1,2,2,0,0.000056
2,26,5,21,0.001816
3,5,3,2,0.000264
4,6,3,3,0.000340


### Drop "weak" products
----
With the now calculated support for each product, we can now evaluate and sort the products. This allows us to remove  weak, rarely purchased products.

In [13]:
def drop_products(df, rate=0.8):
    """
    Function drops a predefined percentage of products
    
    :param df: pandas dataframe
    :param rate: float that definies how many products should be dropped (percentage)
    :return df: DataFrame containing all the product_name's to keep
    """
    # sort values:
    df.sort_values('rating', ascending=False, inplace=True)
    
    # calculate condition to drop
    rows_to_drop = int(len(df)*rate)
    df = df.drop(df.tail(rows_to_drop).index)
    
    return df

In [14]:
reduced_df = drop_products(df);

In [15]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6266 entries, 2227 to 10946
Data columns (total 4 columns):
n_orders      6266 non-null int64
n_users       6266 non-null int64
n_reorders    6266 non-null int64
rating        6266 non-null float64
dtypes: float64(1), int64(3)
memory usage: 244.8 KB


In [16]:
reduced_df.head()

,n_orders,n_users,n_reorders,rating
2227,13760,2244,11543,1.000000
2084,11715,1976,9758,0.849907
20229,8300,1877,6451,0.593540
17867,7280,1711,5593,0.519307
18969,6970,1400,5590,0.501686


In [17]:
# list of products to keep:
good_products = reduced_df.index.tolist()

In [18]:
# drop columns with bad products
if Path('rating.csv').is_file():
    pass
else:
    chunks = pd.read_csv('Recommender4Retail.csv', chunksize=10_000)
    for chunk in chunks:
        chunk = chunk[chunk['product_name'].isin(good_products)]

        if Path('rating.csv').is_file():
            chunk.to_csv('rating.csv', mode='a', header=False)
        else:
            chunk.to_csv('rating.csv', mode='a', header=True)

## User reduction using "number of orders"
----
Many users only use a Service once in their lifetime. Now this is a potential problem. How should we recommend a Product to a user that we don't know. One Option, which is always available, is to just recommend the most commen products. We see this in onlinestores under the tag 'Most popular products' or in rankings like 'Top 10 products for [insert activity here]'. This is fairly easy to implement. The recommender on the other hand can't make much use of these users. To train the recommender on users that actually have some value for the recommender, we drop users below a certain threshold.

In [76]:
df = pd.read_csv('Recommender4Retail.csv', nrows = 1_000_000)
df.head()

,Unnamed: 0,user_id,order_id,product_name,reordered,aisle_id
0,0,1,2539329,Soda,0,77
1,1,1,2539329,Organic Unsweetened Vanilla Almond Milk,0,91
2,2,1,2539329,Original Beef Jerky,0,23
3,3,1,2539329,Aged White Cheddar Popcorn,0,23
4,4,1,2539329,XL Pick-A-Size Paper Towel Rolls,0,54


In [77]:
#group by user_id and count number of orders
df_grouped = df.groupby('user_id').agg({'order_id': 'nunique'}).reset_index()
print(f'Length of grouped DataFrame: {len(df_grouped)}')
df_grouped.head()

Length of grouped DataFrame: 6212


,user_id,order_id
0,1,11
1,2,15
2,3,12
3,4,5
4,5,5


In [100]:
#drop customer with order_id count below n:
n = 8
df_user = df_grouped.loc[df_grouped.order_id >= n]
print(f'Length of grouped DataFrame: {len(df_user)}')

Length of grouped DataFrame: 3855


In [101]:
# drop transactions
print(f"Length DataFrame before: {len(df)}")
users = df_user.user_id.to_list()
df_selected = df[df.user_id.isin(users)]
print(f"Length DataFrame after: {len(df_selected)}")

Length DataFrame before: 1000000
Length DataFrame after: 886136


In [103]:
df_selected.head()

,Unnamed: 0,user_id,order_id,product_name,reordered,aisle_id
0,0,1,2539329,Soda,0,77
1,1,1,2539329,Organic Unsweetened Vanilla Almond Milk,0,91
2,2,1,2539329,Original Beef Jerky,0,23
3,3,1,2539329,Aged White Cheddar Popcorn,0,23
4,4,1,2539329,XL Pick-A-Size Paper Towel Rolls,0,54
